<a href="https://colab.research.google.com/github/GambimL/Card_de_login/blob/master/mask_extration(Vestibular-Schwannoma%20SEG%20Dataset).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Download libraries

In [2]:
!pip install rt_utils #this lib is for reconstruct the RTstruct of planing
!pip install nibabel #this lib is for save arhives in nii file format

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 8.2 MB/s eta 0:00:00


conection with drive


In [1]:
from google.colab import drive #mount the drive of user
drive.mount('/content/drive')

Mounted at /content/drive


load libraries

In [3]:
from rt_utils import RTStructBuilder
import matplotlib.pyplot as plt
import nibabel as nib
import pydicom as pydcm
import numpy as np
import os

configurations

In [ ]:
plt.rcParams['figure.figsize'] = [3, 3] #configure the size of figure in matplotlib.pyplot

## Data visualization

In [ ]:
#this cell is for visualization of  MRI images

file_path = "/content/drive/MyDrive/VS_seg/dicom_data/vs_gk_39_t1/IMG0000000034.dcm" #path of iage
ds = pydcm.dcmread(file_path) #load dicom file in a varable
img = ds.pixel_array #extracts pixel matrix of dicom file
plt.imshow(img,cmap='gray') #plot the image
plt.show()

In [ ]:
#this cell is etract a mask ande visualize

slicers_path = "/content/drive/MyDrive/VS_seg/dicom_data/vs_gk_40_t1" #slices of mri exam path
rtstruct_path = "/content/drive/MyDrive/VS_seg/dicom_data/vs_gk_40_t1/RTSS.dcm" #RTstruct path


# this method load rt struct and dicom series related in the path
rtstruct = RTStructBuilder.create_from(
  dicom_series_path = slicers_path,
  rt_struct_path = rtstruct_path
)


print(rtstruct.get_roi_names()) #print the roi existing in the rt struct file

mask_3d = rtstruct.get_roi_mask_by_name("AN") #get the masks slices for an roi in the rt struct file

first_mask_slice = mask_3d[:, :, 34] #get the mask 34 in slices masks
plt.imshow(first_mask_slice, cmap='gray')
plt.show()

## Extracting masks in sequence

Path files

In [5]:
output_path = "/content/drive/MyDrive/VS_seg/input_data" #output path of masks

Extracting masks in nifit format


In [12]:
number_of_exams = 250 #number of exams in dataset
number_of_stop = 228 #exam number that the interaction will start

for i in range(number_of_stop, number_of_exams + 1):
  os.mkdir(f"/content/drive/MyDrive/VS_seg/input_data/vs_gk_{i}") #creates the directory where the mask for that interaction will be stored
  dir_name = [f"vs_gk_{i}_t1",f"vs_gk_{i}_t2"]


  for k in range(len(dir_name)):
    #This block occurs the same as in cell 5
    #_____________________________________________________________________
    slicers_path = f"/content/drive/MyDrive/VS_seg/dicom_data/{dir_name[k]}"
    rtstruct_path = f"/content/drive/MyDrive/VS_seg/dicom_data/{dir_name[k]}/RTSS.dcm"


    rtstruct = RTStructBuilder.create_from(
    dicom_series_path = slicers_path,
    rt_struct_path = rtstruct_path
    )

    masks = rtstruct.get_roi_names()
    mask_3d = rtstruct.get_roi_mask_by_name(masks[0])
    #______________________________________________________________________


    data = mask_3d.astype(int) #store the masks matrix in int datatype in a variable
    img = nib.Nifti1Image(data, np.eye(4)) #create a nii image

    nib.save(img, f"{output_path}/vs_gk_{i}/vs_gk_seg_refT{k+1}.nii.gz") #save the nii image


<ipython-input-12-c548a11d2dd4>:24: FutureWarning: Image data has type int64, which may cause incompatibilities with other tools. This will error in NiBabel 5.0. This warning can be silenced by passing the dtype argument to Nifti1Image().
  img = nib.Nifti1Image(data, np.eye(4))
